In [1]:
!pip install emoji

In [2]:
import emoji as emoji #Importing the emoji the library which has certain emojis for certain text

In [3]:

emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }
#Only these are present in dictionary so we created labels for the emojis

In [4]:
emoji.emojize(":downcast_face_with_sweat:") #Testing the function

'😓'

In [5]:
for e in emoji_dictionary.values(): #Printing all the emojis
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [6]:
import numpy as np
import pandas as pd
train = pd.read_csv('train_emoji.csv',header=None) #Downloading train and test data
test = pd.read_csv('test_emoji.csv',header=None)

In [7]:
train.head()
#We can see only 0 and first column is useful

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [8]:
data = train.values
for i in range(10): #First 10 rows
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])])) #Will check each row
    #The first column of each row will be printed
    #Then second column has a label which has some string attached to it, which when used in a function gives us a emoji

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [11]:
f = open('glove.6B.50d.txt',encoding='utf-8')
#Taking the file

In [12]:
embeddings = {}
for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')

        embeddings[word] = coeffs
f.close()
#We got 6 billion words and their coefficients in our dictionary

In [13]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50)) #Create three dimensional matrix
    #There are same number of rows as number of inputs
    #Maximum length of each sentence is 10 so we will store 10
    #Then for each word we will store their 50 embeddings
    
    for ix in range(X.shape[0]):
        
        X[ix] = X[ix].split()
        #All words
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            #For words for which embeddings are there we will post them
            
    return embedding_matrix_output

In [10]:
from keras.utils import to_categorical

XT = train[0] #Will take all the setences
Xt = test[0] #Test set sentences

YT = to_categorical(train[1]) #Convert labels into hot encoder
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

Using TensorFlow backend.


(132,)
(56,)
(132, 5)
(56, 5)


In [14]:
emb_XT = getOutputEmbeddings(XT)#For training
emb_Xt = getOutputEmbeddings(Xt) #For test

C:\Users\Sachin Sourav Jha\Anaconda3\envs\AntonioEnv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [17]:
print(emb_XT.shape)
print(emb_Xt.shape)

(132, 10, 50)
(56, 10, 50)


In [18]:
from keras.layers import * #All the layers
from keras.models import Sequential

In [28]:
model = Sequential() #We will do sequentially
model.add(LSTM(64,input_shape=(10,50),return_sequences=True)) #We will take input and give 64 neurons
#Return_Sequences being true ensures that we get an output which can be input for another LSTM layer
model.add(Dropout(0.4)) #For saving overfitting
model.add(LSTM(64,input_shape=(10,50))) #Then putting it in second LSTM, which is known as stacked LSTM
model.add(Dropout(0.3))
model.add(Dense(5)) #We have give different emojis so we use this 
model.add(Activation('softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [30]:
from keras.callbacks import ModelCheckpoint # save the best model, fight overfiitting
from keras.callbacks import EarlyStopping #save time
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#Will keep checking the val_loss and it will only store the best one

hist = model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1, callbacks=[checkpoint])

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 2s 18ms/step - loss: 1.5666 - acc: 0.3136 - val_loss: 1.6055 - val_acc: 0.2857
Epoch 2/40
118/118 [==============================] - 0s 653us/step - loss: 1.5038 - acc: 0.3051 - val_loss: 1.6393 - val_acc: 0.3571
Epoch 3/40
118/118 [==============================] - 0s 600us/step - loss: 1.4860 - acc: 0.2966 - val_loss: 1.6606 - val_acc: 0.2857
Epoch 4/40
118/118 [==============================] - 0s 684us/step - loss: 1.4413 - acc: 0.4322 - val_loss: 1.6177 - val_acc: 0.2143
Epoch 5/40
118/118 [==============================] - 0s 579us/step - loss: 1.4038 - acc: 0.4322 - val_loss: 1.5501 - val_acc: 0.2857
Epoch 6/40
118/118 [==============================] - 0s 651us/step - loss: 1.3255 - acc: 0.5508 - val_loss: 1.4820 - val_acc: 0.2857
Epoch 7/40
118/118 [==============================] - 0s 657us/step - loss: 1.2479 - acc: 0.5932 - val_loss: 1.4126 - val_acc: 0.3571
Epoch 8/40
118/118

In [31]:
model.evaluate(emb_Xt,Yt) #Will give best accuracy
#Low accuracy is because the emojis are similar as we can check
#Smile and heart emojis can be used interchangebly so it is ok


56/56 [==============================] - 0s 285us/step


[1.8381744623184204, 0.5892857228006635]